In [ ]:
links = {"Nicos Weg A1 (English)": "https://learngerman.dw.com/en/nicos-weg/c-36519789",
         "Nicos Weg A2 (English)": "https://learngerman.dw.com/en/nicos-weg/c-36519797",
         "Nicos Weg B1 (Deutsch)": "https://learngerman.dw.com/de/nicos-weg/c-36519718"}

In [ ]:
!pip install selenium
!pip install beautifulsoup4
!pip install genanki requests
!pip install pyvirtualdisplay
!apt-get update
!sudo apt-get install -y xvfb
!apt install -y chromium-chromedriver

In [ ]:
from pyvirtualdisplay import Display
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
def get_links(urls):
    display = Display(visible=0, size=(1920, 1080))
    display.start()
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options=options)
    data = {}
    for url in urls:
        print(url)
        data[url] = {}
        driver.get(urls[url])
        containers = WebDriverWait(driver, 20).until(ec.presence_of_all_elements_located((By.CLASS_NAME, "l1qg9c3h")))
        for container in containers:
            driver.execute_script("arguments[0].scrollIntoView(true);", container)
            section_title = container.find_element(By.XPATH, value="div/div/h2").text
            data[url][section_title] = {}
            lessons = WebDriverWait(container, 20).until(ec.presence_of_all_elements_located((By.CLASS_NAME, "l1cmdjmq")))
            for lesson in lessons:
                lesson_url = lesson.find_element(By.TAG_NAME, "a").get_attribute("href")
                lesson_titl = lesson.find_element(By.XPATH, value="a/div[1]/h3")
                html_string = lesson_titl.get_attribute("outerHTML")
                soup = BeautifulSoup(html_string, 'html.parser')
                lesson_title = soup.get_text()
                learning_targe = lesson.find_element(By.XPATH, value="a/div[1]/span[1]")
                html_string = learning_targe.get_attribute("outerHTML")
                soup = BeautifulSoup(html_string, 'html.parser')
                learning_target = soup.get_text()
                try:
                    categorie = lesson.find_element(By.XPATH, value="a/div[1]/span[2]")
                    html_string = categorie.get_attribute("outerHTML")
                    soup = BeautifulSoup(html_string, 'html.parser')
                    categories = soup.get_text()
                    if categories == '':
                        categories = "N/A"
                except NoSuchElementException:
                    categories = "N/A"
                data[url][section_title][lesson_title] = {
                    "lesson_url": lesson_url,
                    "lesson_target": learning_target,
                    "categories": categories
                }
        data[url].popitem()
    driver.quit()
    return data

In [ ]:
import json
with open("lessonlinks.json", "w", encoding="utf-8") as file:
    data = get_links(links)
    json.dump(data, file, indent=4, ensure_ascii=False)
    print("lessonlinks.json saved successfully")

In [ ]:
def vc_scrape():
    with open('lessonlinks.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    display = Display(visible=0, size=(1920, 1080))
    display.start()
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options=options)
    for level in data:
        print(level)
        for section in data[level]:
            print("   ", section)
            for lesson in data[level][section]:
                print("       ", lesson)
                data[level][section][lesson]["vocabularies"] = []
                driver.get(data[level][section][lesson]["lesson_url"]+"/lv")
                containers = WebDriverWait(driver, 20).until(ec.presence_of_all_elements_located((By.CLASS_NAME, "srr46ge")))
                for container in containers:
                    driver.execute_script("arguments[0].scrollIntoView(true);", container)
                    try:
                        german_word = container.find_element(By.XPATH, value="div[1]/a/strong").text
                    except NoSuchElementException:
                        german_word = container.find_element(By.XPATH, value="div[1]/strong").text
                    try:
                        audio_link = container.find_element(By.XPATH, value="div[1]/a/audio/source").get_attribute("src")
                    except NoSuchElementException:
                        audio_link = "N/A"
                    try:
                        note = container.find_element(By.XPATH, value="div[1]/span").text
                        if note == '':
                            note = "N/A"
                    except NoSuchElementException:
                        note = "N/A"
                    try:
                        image_link = container.find_element(By.XPATH, value="div[2]/figure/img[1]").get_attribute("src")
                        if image_link == '':
                            image_link = "N/A"
                    except NoSuchElementException:
                        image_link = "N/A"
                    try:
                        translation = container.find_element(By.XPATH, value="div[3]/span/p").text
                    except NoSuchElementException:
                        translation = container.find_element(By.XPATH, value="div[3]/span/table/tbody/tr/td").text
                    data[level][section][lesson]["vocabularies"].append({
                        "german_word": german_word,
                        "audio_link": audio_link,
                        "image_link": image_link,
                        "translation": translation,
                        "note": note
                    })
    driver.quit()
    return data

In [ ]:
with open("vocabularylinks.json", "w" , encoding="utf-8") as file:
    data = vc_scrape()
    json.dump(data, file, indent=4, ensure_ascii=False)
    print("vocabularylinks.json saved")

In [ ]:
import json
import re
import os
import requests
from urllib.parse import urlparse
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import genanki

In [ ]:
media_dir = 'media'
os.makedirs(media_dir, exist_ok=True)

def download_media(url, directory):
    if url == "N/A":
        return None

    try:

        # Generate filename
        filename = os.path.basename(urlparse(url).path).lower()
        filepath = os.path.join(directory, filename)

        # If file exists, modify filename
        if os.path.exists(filepath):
            return filename
            # filename = f"R_{filename}"
            # filepath = os.path.join(directory, filename)

        # Configure retry strategy
        retry_strategy = Retry(
            total=300,
            backoff_factor=1,
            status_forcelist=[500, 502, 503, 504],
            allowed_methods=["GET"]
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)

        # Create persistent session
        with requests.Session() as session:
            session.mount("https://", adapter)
            session.mount("http://", adapter)

            # Stream download with timeout
            response = session.get(url, stream=True, timeout=(3.05, 30))
            response.raise_for_status()

            # Download in chunks
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:  # filter out keep-alive chunks
                        f.write(chunk)

            # Verify file integrity
            content_length = response.headers.get('Content-Length')
            if content_length:
                actual_size = os.path.getsize(filepath)
                if actual_size != int(content_length):
                    os.remove(filepath)  # Remove corrupted file
                    raise IOError(f"File size mismatch: {actual_size} vs {content_length}")

            return filename

    except Exception as e:
        print(f"Error downloading {url}: {e}")
        # Clean up potentially corrupted file
        if os.path.exists(filepath):
            try:
                os.remove(filepath)
            except OSError as cleanup_error:
                print(f"Error cleaning up file: {cleanup_error}")
        return None


def form_name(idn, name):
    # Remove leading numbers from course_name
    cleaned_name = re.sub(r'^\d+\s*', '', name)

    # Format index as a two-digit number
    return f"{idn:02} {cleaned_name}"

model = genanki.Model(
    1607392323,
    'Nicos Weg Model',
    fields=[
        {'name': 'GermanWord'},
        {'name': 'Translation'},
        {'name': 'Audio'},
        {'name': 'Image'},
        {'name': 'LessonTitle'},
        {'name': 'LessonURL'},
        {'name': 'LearningTarget'},
        {'name': 'Grammar'},
        {'name': 'Notes'},
    ],
    templates=[
        {
            'name': 'Translation-Word',
            'qfmt': '''
<meta charset='UTF-8'>
<div class="card">
    <div class="word" {{#Image}}, style="border-radius: 10px 10px 0 0;"{{/Image}}>
    {{Translation}}
</div>
{{#Image}}
<div class="image-container">{{Image}}</div>
{{/Image}}
</div>
''',
            'afmt': '''
<meta charset='UTF-8'>
<div class="card", style="border-radius: 10px 10px 0 0;">
    <div class="word", style="border-radius: 10px 10px 0 0;">
        {{Translation}}
    </div>
    {{#Image}}
    <div class="image-container", style="border-radius: 0 0 0 0;">{{Image}}</div>
    {{/Image}}
</div>
<div class="card", style="border-radius: 0 0 10px 10px;">
    <div class="translation">{{GermanWord}}</div>
    <div class="audio">
        {{Audio}}
    </div>
    <div style="text-align: left; margin-top: 7px; padding-left: 10px; padding-bottom: 5px; font-family: Arial;">
        <div><b>Lesson:</b> {{LessonTitle}}</div>
        <div><b>Learning Target:</b> {{LearningTarget}}</div>
        {{#Grammar}}
        <div><b>Grammar:</b> {{Grammar}}</div>
        {{/Grammar}}
        {{#Notes}}
        <div><b>Notes:</b> {{Notes}}</div>
        {{/Notes}}
        <div><a href="{{LessonURL}}">Lesson Link</a></div>
    </div>
</div>
''',
        },
    ],
    css='''
.card {
    font-family: Arial;
    font-size: 24px;
    text-align: center;
    background-color: #98c1d9;
    border-radius: 10px;
}

.word {
    background-color: #3d5a80;
    color: #ffffff;
    padding: 7px;
    border-radius: 10px 10px 10px 10px;
}

.audio {
    padding: 5px;
    background-color: #98c1d9;
    border-radius: 0 0 10px 10px;
}

.translation {
    font-size: 24px;
    color: #ffffff;
    padding: 7px;
    text-align: center;
}

a {
    color: #00308F;
    text-decoration: None;
}

.image-container {
    overflow: hidden;
    border-radius: 0 0 10px 10px;
}

.image-container img {
    display: block;
    width: 100%;
    object-fit: cover;
}
'''
)

# Load the JSON data
with open('vocabularylinks.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

decks = []
media_files = []

for course_name, course_data in data.items():
    print(course_name)
    for idc, (section_name, section_data) in enumerate(course_data.items()):
        print("   ", section_name)
        for idl, (lesson_name, lesson_data) in enumerate(section_data.items(), start=1):
            print("      ", lesson_name)
            scn_name = form_name(idc, section_name)
            lsn_name = f"{idl} {lesson_name}"
            deck_path = f"DW German Courses::{course_name}::{scn_name}::{lsn_name}"
            deck_id = abs(hash(deck_path)) % (10**10)  # Unique ID
            deck = genanki.Deck(deck_id, deck_path)
            decks.append(deck)
            # Add vocabulary notes to the deck
            for vocabulary in lesson_data['vocabularies']:
                # Download media
                audio_filename = download_media(vocabulary["audio_link"], media_dir)
                image_filename = download_media(vocabulary['image_link'].replace("_600.", "_607."), media_dir)

                # Collect media paths
                if audio_filename:
                    media_files.append(os.path.join(media_dir, audio_filename))
                if image_filename:
                    media_files.append(os.path.join(media_dir, image_filename))
                # Create note
                note = genanki.Note(
                    model=model,
                    fields=[
                        vocabulary["german_word"],
                        vocabulary["translation"].replace('\n', '<br>'),
                        f'[sound:{audio_filename}]' if audio_filename else '',
                        f'<img src="{image_filename}">' if image_filename else '',
                        lesson_name,
                        lesson_data['lesson_url'],
                        lesson_data['lesson_target'],
                        lesson_data['categories'] if lesson_data['categories'] != "N/A" else '',
                        vocabulary['note'].replace('\n', '<br>') if vocabulary['note'] != "N/A" else '',
                    ]
                )
                deck.add_note(note)


package = genanki.Package(decks)
package.media_files = media_files
package.write_to_file('DW_German_Nico_A1-B1.apkg')

print("APKG file created successfully!")